In [1]:
from transformers import pipeline

import os
import cv2
import numpy as np
import glob
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from PIL import Image

import segmentation_models as sm
from tensorflow.keras.metrics import MeanIoU
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import random

from transformers import DPTFeatureExtractor, DPTForDepthEstimation
import torch
import numpy as np
from PIL import Image
import requests

import pandas as pd

import pickle

import re
import os
import cv2
import numpy as np
import pandas as pd
import glob
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
from sklearn.model_selection import train_test_split
from PIL import Image

import segmentation_models as sm
from tensorflow.keras.metrics import MeanIoU
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import random
import torch
from PIL import Image,ImageFilter
import requests

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2


Segmentation Models: using `keras` framework.


In [5]:
#these codes are used to loop through the directory
path_images = 'rs19_val/jpgs/rs19_val/'
path_images_resized = 'rs19_val/960_512/images_resized/'
path_masks = 'rs19_val/uint8/rs19_val/'
path_masks_resized = 'rs19_val/960_512/masks_resized_track/'


# read the tram list and turn it into a dict
tram_list = pd.read_csv('tram_filenames.csv')
tram_dict = tram_list.iloc[:,1:].to_dict()

In [6]:
# pixel labels for each class
tram_track=3 
rail_track=12
track_bed= 15
rail_raised= 17
rail_embedded =18
veg = 8

# Prepare the mask label containing 5 labels related to tracks

- 200 tram images 
- Input images and Masks -> resized (NN) 512x256 , Median filter
- Prepare Binary Masks for all 5 options of labels related to track identification

In [10]:
image_dataset_resized = []
masked_dataset_resized = []

down_width = 960
down_height = 512
down_points = (down_width, down_height)

for i in range(len(tram_list)):
    image_tram = cv2.imread(path_images+tram_dict['filename_jpg'][i],1)
    mask_tram =  cv2.imread(path_masks+tram_dict['filename_mask'][i],0)
    # Median filter 3x3 and resizing 960x512 (such that the dimensions are divisible by 32 becuase of model requiremnet)
    #resize the input image by a scale factor using k-nearest interpolation such that the dimensions are divisible by 32
    #image_tram= image_tram.filter(ImageFilter.MedianFilter(size= 3))
    image_tram = cv2.medianBlur(image_tram, 3)
    image_resized= cv2.resize(image_tram, down_points, interpolation= cv2.INTER_NEAREST)

    cv2.imwrite(path_images_resized+tram_dict['filename_jpg'][i]+'.jpg', image_resized)
    image_dataset_resized.append(image_resized)

    #repeat for masks
    #mask_tram= mask_tram.filter(ImageFilter.MedianFilter(size= 3))
    mask_tram=cv2.medianBlur(mask_tram, 3)
    mask_resized= cv2.resize(mask_tram, down_points, interpolation= cv2.INTER_NEAREST)
 
    # Binary masks for track
    for k,val in enumerate(mask_resized):
        for j,val1 in enumerate(val):
            if (val1==tram_track or val1== rail_track or val1==rail_embedded or val1==rail_raised or val1== track_bed):
                val[j] = 1
            else:
                val[j] = 0
    
    masked_dataset_resized.append(mask_resized)
    cv2.imwrite(path_masks_resized+tram_dict['filename_mask'][i]+'.png', mask_resized)

image_dataset_resized = np.array(image_dataset_resized)
masked_dataset_resized = np.array(masked_dataset_resized)

#pickle.dump(image_dataset_resized, open('image_dataset_resized_10', "wb"))
#pickle.dump(masked_dataset_resized, open('mask_dataset_10', "wb"))
#pickle.dump(dataset_mask_multiplied, open('dataset_roi_mask_10', "wb"))

print(image_resized.shape)
print(mask_resized.shape)
print(image_dataset_resized.shape)
print(masked_dataset_resized.shape)

In [ ]:
test_list = ['rs00079','rs01562','rs02536','rs03764','rs04744','rs05410','rs07603','rs08095']

In [ ]:
down_width = 960
down_height = 512
down_points = (down_width, down_height)

for imgfile in range(len(test_list)):
    chk_img= cv2.imread(path_images_resized+test_list[imgfile]+'.jpg')
    
    chk_msk= cv2.imread(path_masks_resized+test_list[imgfile]+'.jpg')


    org_msk = cv2.imread(path_masks+test_list[imgfile]+'png',0)
    org_msk = cv2.medianBlur(org_msk,3)
    org_msk = cv2.resize(org_msk, down_points, interpolation= cv2.INTER_NEAREST)

    figure, axis_arr = plt.subplots(1,3, figsize=(20,20)) 
    axis_arr[0].imshow(chk_img)
    axis_arr[0].get_title('image_resized')
    axis_arr[1].imshow(org_msk, cmap='prism')
    axis_arr[1].get_title('mask_original_resized')
    axis_arr[2].imshow(chk_msk, cmap='prism')
    axis_arr[2].get_title('mask_resized_track')